In [165]:
import pandas as pd    # carrega a biblioteca pandas (que permite manipular data-frames em memória)
import numpy as np
import random
import requests        # biblioteca que consegue ler arquivos que estão na Web (em uma URL específica)
from queue import PriorityQueue
from dataclasses import dataclass, field

np.set_printoptions(threshold=np.inf)

filename = 'instancia_50_100_1.dat' 
uri = 'https://raw.githubusercontent.com/ricarditt0/Trabalho-Otimizacao-2022-2/main/' + filename
  
content = requests.get(uri).content
content = (content.decode('utf-8'))
content = content.splitlines()
tasks = {}
n ,C = content[0].split()
n = int(n)
C = int(C)
del content[0]
tmax=0
for con in content:
  key, s ,t ,c = con.split()
  if tmax < int(t):
    tmax = int(t)
  tasks.update({int(key):(int(s),int(t),int(c))})

@dataclass(order=True)
class Chomossome:
    score: float
    genome: np.array = field(compare=False)

def evaluation(genome,list_task=tasks,restriction=C):
  size_t = len(genome)
  resorce = np.array(np.zeros((size_t,tmax+1)))
  schedule = {}
  acumulate = 0

  for i in genome:
    j = 0 
    alloc = False
    while not alloc:
      if (not schedule):
        schedule.update({j:set()})
        schedule[j].add(i)
        resorce[j][list_task[i][0]:list_task[i][1]+1] += list_task[i][2]
        alloc = True
      else:
        resorce_aux = np.copy(resorce)
        resorce_aux[j][list_task[i][0]:list_task[i][1]+1] += list_task[i][2]
        resorce_aux = resorce_aux[j][list_task[i][0]:list_task[i][1]+1] <= restriction
        if np.all(resorce_aux):
          schedule[j].add(i)
          resorce[j][list_task[i][0]:list_task[i][1]+1] += list_task[i][2]
          alloc = True
        else:
          if j == list(schedule.keys())[-1]:
            j += 1
            schedule.update({j:set()})
          else:
            j += 1

  return(len(schedule))

#reconbinação em um ponto
def crossing_over(genome1,genome2):
  point = random.randint(2,len(genome1)-3)
  decison = random.randint(0,1)
  
  g1_0 = genome1[:point]
  g1_1 = genome1[point:]
  g2_0 = genome2[:point]
  g2_1 = genome2[point:]
  son0 = np.array(np.zeros(len(genome1)))
  son1 = np.array(np.zeros(len(genome1)))
  if decison == 0:
    son0 = np.concatenate((g1_0,genome2))
    son1 = np.concatenate((g2_1,genome1))
  else :
    son0 = np.concatenate((g2_0,genome1))
    son1 = np.concatenate((g1_1,genome2))

  son0 = np.array(list(dict.fromkeys(son0)))
  son1 = np.array(list(dict.fromkeys(son1)))
  return(son0,son1)

def init_pop(pop_size ,genome_size = n):
  pop = PriorityQueue()
  for i in range(0,pop_size): 
    genome = np.array(random.sample(range(0,genome_size),k = genome_size))
    a = evaluation(genome)
    pop.put(Chomossome(a,genome))
  return pop

def mutation(genome,mutation_rate):
  choice = random.choices([0,1], weights=[100-mutation_rate,mutation_rate], k=1)
  if choice[0] == 1:
    quant_genes = max(2,(len(genome)//10))
    index = np.array(random.sample(range(0,len(genome)),k = quant_genes))
    genes = np.array(np.zeros(quant_genes),dtype='i')
    for i in range(0,quant_genes):
      genes[i] = genome[index[i]]
    random.shuffle(genes)
    for i in range(0,quant_genes):
      genome[index[i]] = genes[i]
    return genome
  else:
    return genome

#Estratégia (λ + µ):
def new_generation(pop,pop_size,mutation_rate):
  weight = np.array(np.zeros(pop_size))
  for i in range(0,pop_size):
   weight[i] = C*1000 - pop.queue[i].score
  for i in range(0,pop_size//2):
    index = (0,0)
    while True:
      index = random.choices(range(0,pop_size), weights=weight, k=2)
      if index[0] != index[1]:
        break
    parent1 = pop.queue[index[0]]
    parent2 = pop.queue[index[1]]
    gen_son1, gen_son2 = crossing_over(parent1.genome,parent2.genome)

    gen_son1 = mutation(gen_son1,mutation_rate)
    gen_son2 = mutation(gen_son2,mutation_rate)

    score = evaluation(gen_son1)
    son1 = Chomossome(score,gen_son1)
    score = evaluation(gen_son2)
    son2 = Chomossome(score,gen_son2) 

    pop.put(son1)
    pop.put(son2)

  return selection(pop, pop_size)

def selection(pop, pop_size):
  new_pop = PriorityQueue()
  for i in range(0,pop_size):
    aux = pop.get()
    new_pop.put(aux)
  return new_pop

def best_individual(pop, best, i):
  individual = pop.get()
  pop.put(individual)
  if individual.score < best.score:
    best = individual
    print(i)
    print(best)
  return best

def genetic_algorithm(pop_size = 50,mutation_rate = 10,generarions = 100):
   best = Chomossome(100,np.array(np.zeros(2)))
   pop = init_pop(pop_size)
   best = best_individual(pop, best,0)
   for i in range(0,generarions):
    pop = new_generation(pop,pop_size,mutation_rate)
    best = best_individual(pop, best,i)
   return(best)

print(genetic_algorithm(50,mutation_rate = 10))

Chomossome(score=20, genome=array([ 0, 30, 16, 46, 19, 32, 26, 47,  1, 39, 33, 23, 11, 34,  8, 29, 43,
       38, 24, 13, 40, 22,  2, 41, 17, 28, 31,  9, 49,  6, 25,  5, 42, 10,
       18, 48,  3, 15, 36, 37,  4, 20, 35, 45, 44, 27, 21,  7, 14, 12]))
